In [2]:
import numpy as np

from bwb.distributions.data_loaders import DistributionDrawDataLoader
from bwb.distributions import ExplicitPosteriorPiN

expl_posterior = ExplicitPosteriorPiN()
expl_posterior

In [3]:
%%time

models_array = np.load(r"..\data\face.npy")
n_faces, _ = models_array.shape
print(f"{n_faces = }")

faces = DistributionDrawDataLoader(models_array, (28, 28))

n_faces = 161666
CPU times: total: 9.42 s
Wall time: 3.4 s


In [4]:
faces[0]

In [5]:
x = faces[0].sample((10,))
x

tensor([189, 529, 276, 122, 663, 160, 555, 366, 157, 295])

In [6]:
%%time
expl_posterior.fit(x, faces)

CPU times: total: 109 ms
Wall time: 49 ms


In [7]:
expl_posterior.rvs(size=10_000, seed=42)

In [8]:
expl_posterior.total_time

0.54784529999597

In [9]:
expl_posterior2 = ExplicitPosteriorPiN()
expl_posterior2.total_time, expl_posterior.total_time

(0.0, 0.54784529999597)

In [10]:
try:
    expl_posterior2.draw()
except Exception as e:
    print(e)

This ExplicitPosteriorPiN instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.


In [11]:
len(expl_posterior.samples_counter)

1150

In [12]:
print(expl_posterior.samples_counter.total())
expl_posterior.draw()
print(expl_posterior.samples_counter.total())

10000
10001
